# Importamos las librerías que vamos a necesitar

In [1]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

%matplotlib inline

# Análisis exploratorio
## Leer e inspeccionar el conjunto de datos

In [2]:
# No es un archivo CSV sino una hoja de Excel
cs_df = pd.read_excel(io=r'2 - MLII - Clase 11 - Datos.xlsx')

In [3]:
cs_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


- InvoiceNo: Un identificador único para la factura. Un número de factura compartido entre
filas significa que esas transacciones se realizaron en una sola factura (múltiples
compras).
- StockCode: Identificador de los artículos contenidos en una factura.
- Description: Descripción textual de cada uno de los artículos de stock.
- Quantity: La cantidad del artículo comprado.
- InvoiceDate: Fecha de la compra.
- UnitPrice: Valor de cada artículo.
- CustomerID: Identificador del cliente que realiza la compra.
- Country: País del cliente.


Número de transacciones

In [4]:
cs_df.shape

(541909, 8)

## Mayores ventas por país

In [5]:
cs_df.Country.value_counts().reset_index().head(n=10)

,Country,count
0,United Kingdom,495478
1,Germany,9495
2,France,8557
3,EIRE,8196
4,Spain,2533
5,Netherlands,2371
6,Belgium,2069
7,Switzerland,2002
8,Portugal,1519
9,Australia,1259


A continuación, podríamos estar interesados en saber cuántos clientes únicos tiene el minorista y el número de pedidos que realizan. También nos interesa saber qué porcentaje de pedidos es realizado por los 10 principales clientes del minorista. Esta información es interesante ya que nos diría si labase de usuarios de la empresa está distribuida de forma relativamente uniforme.

In [6]:
cs_df.CustomerID.unique().shape

(4373,)

In [7]:
(cs_df.CustomerID.value_counts()/sum(cs_df.CustomerID.value_counts())*100).head(n=13).cumsum()

CustomerID
17841.0    1.962249
14911.0    3.413228
14096.0    4.673708
12748.0    5.814728
14606.0    6.498553
15311.0    7.110850
14646.0    7.623350
13089.0    8.079807
13263.0    8.492020
14298.0    8.895138
15039.0    9.265809
14156.0    9.614850
18118.0    9.930462
Name: count, dtype: float64

## Calidad de datos





Número de artículos únicos

In [8]:
cs_df.StockCode.unique().shape

(4070,)

Descripción de los artículos: Vemos que las descripciones son más que los códigos de stock por lo que debe haber algún código de stock que tenga más de una descripción

In [9]:
cs_df.Description.unique().shape

(4224,)

In [10]:
cs_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [11]:
cat_des_df = cs_df.groupby(["StockCode","Description"]).count().reset_index()

Códigos de stock que tienen más de una descripción

In [12]:
cat_des_df.StockCode.value_counts()[cat_des_df.StockCode.value_counts()>1].reset_index().head()

,StockCode,count
0,20713,8
1,23084,7
2,21830,6
3,85175,6
4,85172,5


Ejemplo de uno de esos códigos

In [13]:
cs_df[cs_df['StockCode'] == cat_des_df.StockCode.value_counts()[cat_des_df.StockCode.value_counts()>1]
      .reset_index()['index'][5]]['Description'].unique()

KeyError: 'index'

In [ ]:
cs_df['invdatetime'] = pd.to_datetime(cs_df.InvoiceDate) # paso los valores de fecha al formato correcto

In [ ]:
cs_df.Quantity.describe()

In [ ]:
cs_df.UnitPrice.describe()

Podemos observar en el resultado anterior que ambos atributos tienen valores negativos,
lo que puede significar que podemos tener algunas transacciones de retorno en nuestros datos también. Este escenario es bastante común
para cualquier minorista, pero tenemos que manejarlos antes de proceder a nuestro análisis. Estos son algunos de los
problemas de calidad de datos que encontramos en nuestro conjunto de datos. En el mundo real, los conjuntos de datos suelen estar desordenados y tienen problemas considerables, por lo que siempre es una buena práctica verificar explícitamente la información antes de realizar cualquier tipo de análisis.


## Estrategia de agrupación (clustering)
Ahora que tenemos alguna información sobre lo que es la segmentación de clientes, varias estrategias y cómo puede
ser útil, podemos empezar con el proceso de búsqueda de segmentos de clientes en nuestro conjunto de datos de venta online.

El conjunto de datos que tenemos consiste únicamente en las transacciones de venta de los clientes y ninguna otra información sobre ellos, es decir, ningún otro atributo adicional. Por lo general, en las organizaciones más grandes solemos tener más atributos
de información sobre los clientes que pueden ayudar a la agrupación. Sin embargo, será interesante
trabajar con este conjunto de datos de atributos limitados. 

Utilizaremos un modelo basado en el valor monetario, la frecuencia y el comportamiento reciente de los clientes (RFM).

El modelo RFM es un modelo popular en marketing y segmentación de clientes para determinar el valor de un cliente. Toma las transacciones de un cliente y calcula tres importantes atributos informativos:

- Recencia: El valor de lo reciente que ha sido la compra de un cliente en el establecimiento
- Frecuencia: La frecuencia de las transacciones del cliente en el establecimiento
- Valor monetario: El valor en dólares (o libras en nuestro caso) de todas las transacciones que
el cliente realizó en el establecimiento

Una combinación de estos tres valores puede servir para asignar un valor al cliente. Podemos pensar directamente
en algunos segmentos deseables que querríamos en dicho modelo. Por ejemplo, un cliente de alto valor es aquel
que compra con frecuencia, que acaba de comprar algo recientemente y que gasta una cantidad elevada cada vez que compra o va de compras.

# Limpieza de datos

In [ ]:
# Separo los datos para un país
cs_df = cs_df[cs_df.Country == 'United Kingdom']

# Creo un nuevo atributo para el monto total
cs_df['amount'] = cs_df.Quantity*cs_df.UnitPrice

# Remuevo transacciones negativas o devoluciones
cs_df = cs_df[~(cs_df.amount<0)]
cs_df.head()
cs_df = cs_df[~(cs_df.CustomerID.isnull())]

In [ ]:
cs_df.shape

## Construyo la característica "Recencia"

In [ ]:
cs_df.InvoiceDate.max()

In [ ]:
cs_df.InvoiceDate.min()

In [ ]:
refrence_date = cs_df.InvoiceDate.max()
refrence_date = refrence_date + datetime.timedelta(days = 1)

In [ ]:
cs_df['days_since_last_purchase'] = refrence_date - cs_df.InvoiceDate
cs_df['days_since_last_purchase_num'] = cs_df['days_since_last_purchase'].astype('timedelta64[D]')


Período de tiempo de las transacciones


In [ ]:
customer_history_df = cs_df.groupby(["CustomerID"])[['days_since_last_purchase_num']].min().reset_index()

In [ ]:
customer_history_df.rename(columns={'days_since_last_purchase_num':'recency'}, inplace=True)


In [ ]:
customer_history_df.recency.describe()

In [ ]:
customer_history_df.head()

In [ ]:
x = customer_history_df.recency
mu = np.mean(customer_history_df.recency)
sigma = math.sqrt(np.var(customer_history_df.recency))
n, bins, patches = plt.hist(x, 1000, facecolor='green', alpha=0.75)

plt.xlabel('Recency in days')
plt.ylabel('Number of transactions')
plt.title(r'$\mathrm{Histogram\ of\ sales\ recency}\ $')
plt.grid(True)

# Construyo las características "Frecuencia" y "Valor monetario"



In [ ]:
customer_monetary_val = cs_df[['CustomerID', 'amount']].groupby("CustomerID").sum().reset_index()
customer_history_df = customer_history_df.merge(customer_monetary_val, how='outer')


In [ ]:
customer_history_df.amount = customer_history_df.amount+0.001
customer_freq = cs_df[['CustomerID', 'amount']].groupby("CustomerID").count().reset_index()
customer_freq.rename(columns={'amount':'frequency'},inplace=True)
customer_history_df = customer_history_df.merge(customer_freq, how='outer')

In [ ]:
customer_history_df.head()

# Preprocesamiento de datos
Una vez que hayamos creado nuestro *dataframe* de valor del cliente, realizaremos un preprocesamiento de los datos. 
Para nuestro clustering, utilizaremos el algoritmo de clustering K-means, del que ya hablamos anteriormente.
Uno de los requisitos para el buen funcionamiento del algoritmo es el centrado de la media de los valores de las variables. Esto significa que sustituiremos el valor real de la variable por un
valor estandarizado, de modo que la variable tenga una media de 1 y una varianza de 0. Esto garantiza que todas las variables
están en el mismo rango y la diferencia de rangos de valores no hace que el algoritmo no funcione bien.

Otro problema sobre el que se puede investigar es el enorme rango de valores que puede tomar cada variable. Este
problema es particularmente notable para la variable de la cantidad monetaria. Para solucionar este problema, vamos a
transformar todas las variables en la escala logarítmica. Esta transformación, junto con la normalización, garantizará
que la entrada a nuestro algoritmo sea un conjunto homogéneo de valores escalados y transformados.

Un punto importante sobre el paso de preprocesamiento de datos es que a veces necesitamos que sea reversible. En
nuestro caso, tendremos los resultados de la agrupación en términos de la variable transformada y escalada. Pero para hacer inferencias en términos de los datos originales, tendremos que invertir la transformación de todas las variables para obtener
las cifras reales del RFM. Esto puede hacerse utilizando las capacidades de preprocesamiento de Python.

In [ ]:
from sklearn import preprocessing
import math

customer_history_df['recency_log'] = customer_history_df['recency'].apply(math.log)
customer_history_df['frequency_log'] = customer_history_df['frequency'].apply(math.log)
customer_history_df['amount_log'] = customer_history_df['amount'].apply(math.log)
feature_vector = ['amount_log', 'recency_log','frequency_log']

X_subset = customer_history_df[feature_vector].to_numpy()

scaler = preprocessing.StandardScaler().fit(X_subset) # usar esto hace la transformación fácilmente reversible
X_scaled = scaler.transform(X_subset)

# Visualización de la Recencia contra el Valor monetario (log)

In [ ]:
plt.scatter(customer_history_df.recency_log, customer_history_df.amount_log, alpha=0.5)

# Visualización de la distribución de "Valor monetario" (log)



In [ ]:
x = customer_history_df.amount_log
n, bins, patches = plt.hist(x, 1000, facecolor='green', alpha=0.75)

plt.xlabel('Log of Sales Amount')
plt.ylabel('Probability')
plt.title(r'$\mathrm{Histogram\ of\ Log\ transformed\ Customer\ Monetary\ value}\ $')
plt.grid(True)
#plt.show()

In [ ]:
customer_history_df.head()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

xs =customer_history_df.recency_log
ys = customer_history_df.frequency_log
zs = customer_history_df.amount_log
ax.scatter(xs, ys, zs, s=5)

ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')

#plt.show()

# Analizamos segmentos de clientes con clustering



# Clustering K-Means
El clustering K-means pertenece a la familia de algoritmos de clustering basados en particiones. Los
pasos que ocurren en el algoritmo K-means para la partición de los datos son los siguientes:

1. El algoritmo comienza con inicializaciones de puntos aleatorios del número requerido de
centros. La "K" en K-means representa el número de clusters.
2. En el siguiente paso, cada uno de los puntos de datos se asigna al centro más cercano a él. La métrica de distancia utilizada en el clustering de K-means es la distancia euclidiana normal.
3. Una vez asignados los puntos de datos, se recalculan los centros promediando las
dimensiones de los puntos pertenecientes al cluster.
4. El proceso se repite con nuevos centros hasta que se llega a un punto en el que las
asignaciones se vuelvan estables. En este caso, el algoritmo termina.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

X = X_scaled

cluster_centers = dict()

for n_clusters in range(3,6,2):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    #ax2 = plt.subplot(111, projection='3d')
    fig.set_size_inches(18, 7)
    ax1.set_xlim([-0.1, 1])
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    silhouette_avg = silhouette_score(X, cluster_labels)
    cluster_centers.update({n_clusters :{
                                        'cluster_center':clusterer.cluster_centers_,
                                        'silhouette_score':silhouette_avg,
                                        'labels':cluster_labels}
                           })

    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        cmap = cm.get_cmap("Spectral")
        color = cmap(float(i) / n_clusters)
        
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    colors = cmap(cluster_labels.astype(float) / n_clusters)
    feature1 = 0
    feature2 = 2
    ax2.scatter(X[:, feature1], X[:, feature2], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')
    
    centers = clusterer.cluster_centers_
    ax2.scatter(centers[:, feature1], centers[:, feature2], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')
    for i, c in enumerate(centers):
        ax2.scatter(c[feature1], c[feature2], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')
    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature i.e. monetary value")
    ax2.set_ylabel("Feature space for the 2nd feature i.e. frequency")
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    #plt.show()

In [ ]:
for i in range(3,6,2):
    print("for {} number of clusters".format(i))
    cent_transformed = scaler.inverse_transform(cluster_centers[i]['cluster_center'])
    print(pd.DataFrame(np.exp(cent_transformed),columns=feature_vector))
    print("Silhouette score for cluster {} is {}". format(i, cluster_centers[i]['silhouette_score']))
    print()

Si observamos los resultados del proceso de agrupación, podemos deducir algunas ideas interesantes. Consideremos
la configuración de tres clústeres y tratemos de entender las siguientes ideas.
- Obtenemos tres clusters con marcadas diferencias en el valor monetario del cliente.
- El clúster 2 es el clúster de los clientes de alto valor que compran con frecuencia y es sin duda
un segmento importante para cada negocio.
- Del mismo modo, obtenemos grupos de clientes con un gasto bajo y medio en
clusters con etiquetas 1 y 0, respectivamente.
- La frecuencia y la recurrencia se correlacionan perfectamente con el valor monetario según la tendencia
(Alta Monetaria-Baja Recencia-Alta Frecuencia).

Los resultados de la configuración de cinco clusters son más sorprendentes. Cuando buscamos más segmentos, descubrimos que nuestra base de clientes de alto valor se compone de dos subgrupos:

- Los que compran a menudo y con un importe elevado (representados por el cluster 0).
- Los que tienen un gasto decente pero no son tan frecuentes (representados por el clúster 1).

Esto entra en conflicto directo con el resultado que obtenemos de la matriz de puntuación de silhouette, que dice que la segmentación de cinco
clústeres es menos óptimos que los segmentos de tres clústeres. 

Por supuesto, recuerde que no siempre debe perseguir estrictamente las métricas matemáticas en todo momento y pensar también en los aspectos comerciales. 


# Asignar etiquetas de cluster

In [ ]:
labels = cluster_centers[5]['labels']   
customer_history_df['num_cluster5_labels'] = labels
labels = cluster_centers[3]['labels']
customer_history_df['num_cluster3_labels'] = labels

In [ ]:
customer_history_df.head()

# Visualizar la segmentación (¿qué conclusiones podemos sacar?


In [ ]:
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode()

x_data = ['Cluster 1','Cluster 2','Cluster 3','Cluster 4', 'Cluster 5']
cutoff_quantile = 100
field_to_plot = 'recency'

y0 = customer_history_df[customer_history_df['num_cluster5_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster5_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster5_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]
y3 = customer_history_df[customer_history_df['num_cluster5_labels']==3][field_to_plot].values
y3 = y3[y3<np.percentile(y3, cutoff_quantile)]
y4 = customer_history_df[customer_history_df['num_cluster5_labels']==4][field_to_plot].values
y4 = y4[y4<np.percentile(y4, cutoff_quantile)]
y_data = [y0,y1,y2,y3,y4]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=50,
        gridcolor='black',
        gridwidth=0.1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [ ]:
x_data = ['Cluster 1','Cluster 2','Cluster 3','Cluster 4', 'Cluster 5']
cutoff_quantile = 80
field_to_plot = 'amount'
y0 = customer_history_df[customer_history_df['num_cluster5_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster5_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster5_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]
y3 = customer_history_df[customer_history_df['num_cluster5_labels']==3][field_to_plot].values
y3 = y3[y3<np.percentile(y3, cutoff_quantile)]
y4 = customer_history_df[customer_history_df['num_cluster5_labels']==4][field_to_plot].values
y4 = y4[y4<np.percentile(y4, cutoff_quantile)]
y_data = [y0,y1,y2,y3,y4]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=1000,
        gridcolor='black',
        gridwidth=0.1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [ ]:
x_data = ['Cluster 1','Cluster 2','Cluster 3','Cluster 4', 'Cluster 5']
cutoff_quantile = 80
field_to_plot = 'frequency'
y0 = customer_history_df[customer_history_df['num_cluster5_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster5_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster5_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]
y3 = customer_history_df[customer_history_df['num_cluster5_labels']==3][field_to_plot].values
y3 = y3[y3<np.percentile(y3, cutoff_quantile)]
y4 = customer_history_df[customer_history_df['num_cluster5_labels']==4][field_to_plot].values
y4 = y4[y4<np.percentile(y4, cutoff_quantile)]
y_data = [y0,y1,y2,y3,y4]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=100,
        gridcolor='black',
        gridwidth=0.1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [ ]:
x_data = ['Cluster 1','Cluster 2','Cluster 3']
cutoff_quantile = 100
field_to_plot = 'recency'
y0 = customer_history_df[customer_history_df['num_cluster3_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster3_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster3_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]

y_data = [y0,y1,y2]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=50,
        gridcolor='black',
        gridwidth=0.1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    plot_bgcolor='white',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [ ]:
x_data = ['Cluster 1','Cluster 2','Cluster 3']
cutoff_quantile = 80
field_to_plot = 'amount'
y0 = customer_history_df[customer_history_df['num_cluster3_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster3_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster3_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]

y_data = [y0,y1,y2]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(        
        dtick=1000,
    )
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)

In [ ]:
x_data = ['Cluster 1','Cluster 2','Cluster 3']
cutoff_quantile = 90
field_to_plot = 'frequency'
y0 = customer_history_df[customer_history_df['num_cluster3_labels']==0][field_to_plot].values
y0 = y0[y0<np.percentile(y0, cutoff_quantile)]
y1 = customer_history_df[customer_history_df['num_cluster3_labels']==1][field_to_plot].values
y1 = y1[y1<np.percentile(y1, cutoff_quantile)]
y2 = customer_history_df[customer_history_df['num_cluster3_labels']==2][field_to_plot].values
y2 = y2[y2<np.percentile(y2, cutoff_quantile)]

y_data = [y0,y1,y2]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)', 'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']
traces = []

for xd, yd, cls in zip(x_data, y_data, colors):
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints=False,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker=dict(
                size=2,
            ),
            line=dict(width=1),
        ))

layout = go.Layout(
    title='Difference in sales {} from cluster to cluster'.format(field_to_plot),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=100,
        gridcolor='black',
        gridwidth=0.1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=False
)

fig = go.Figure(data=traces, layout=layout)
py.offline.iplot(fig)